In [1]:
from flask import Flask, request, jsonify
from threading import Thread
import json
import requests
from datetime import datetime, timedelta
import re
from typing import Dict, List, Tuple, Optional
from dateutil import parser as date_parser
import pytz

In [2]:
app = Flask(__name__)
received_data = []

class AgenticScheduler:
    def __init__(self, vllm_url: str = "http://localhost:3000/v1", model_name: str = "/home/user/Models/deepseek-ai/deepseek-llm-7b-chat"):
        """Initialize the Agentic Scheduler"""
        self.vllm_url = vllm_url
        self.model_name = model_name
        self.timezone = pytz.timezone('Asia/Kolkata')  # IST timezone
        
    def parse_email_content(self, email_content: str, subject: str, attendees: List[str]) -> Dict:
        """Parse email content using LLM to extract meeting requirements"""
        prompt = f"""
        You are an AI scheduling assistant. Parse the following meeting request and extract key information.
        
        Subject: {subject}
        Email Content: {email_content}
        Attendees: {', '.join(attendees)}
        
        Extract and return ONLY a JSON object with:
        1. "meeting_duration_minutes": Duration in minutes (default 60 if not specified)
        2. "time_preference": Specific day/time mentioned or "flexible" 
        3. "urgency": "high", "medium", or "low"
        4. "meeting_type": "status_update", "planning", "review", or "general"
        
        Return only valid JSON, no other text.
        """
        
        try:
            response = requests.post(
                f"{self.vllm_url}/chat/completions",
                json={
                    "model": self.model_name,
                    "messages": [{"role": "user", "content": prompt}],
                    "temperature": 0.1,
                    "max_tokens": 200
                },
                timeout=5
            )
            
            if response.status_code == 200:
                content = response.json()['choices'][0]['message']['content']
                # Extract JSON from response
                json_match = re.search(r'\{.*\}', content, re.DOTALL)
                if json_match:
                    return json.loads(json_match.group())
            
        except Exception as e:
            print(f"LLM parsing error: {e}")
        
        # Fallback parsing
        return self._fallback_parse(email_content, subject)
    
    def _fallback_parse(self, email_content: str, subject: str) -> Dict:
        """Fallback parsing without LLM"""
        duration = 60  # default
        
        # Extract duration from email content
        duration_patterns = [
            r'(\d+)\s*minutes?',
            r'(\d+)\s*mins?',
            r'(\d+)\s*hour?s?',
            r'half\s*hour',
            r'30\s*min'
        ]
        
        for pattern in duration_patterns:
            match = re.search(pattern, email_content.lower())
            if match:
                if 'hour' in pattern:
                    duration = int(match.group(1)) * 60
                elif 'half' in pattern:
                    duration = 30
                else:
                    duration = int(match.group(1))
                break
        
        # Determine urgency
        urgency = "medium"
        if any(word in email_content.lower() for word in ['urgent', 'asap', 'immediately']):
            urgency = "high"
        elif any(word in email_content.lower() for word in ['when convenient', 'flexible', 'whenever']):
            urgency = "low"
        
        # Time preference
        time_pref = "flexible"
        days = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
        for day in days:
            if day in email_content.lower():
                time_pref = day
                break
        
        return {
            "meeting_duration_minutes": duration,
            "time_preference": time_pref,
            "urgency": urgency,
            "meeting_type": "general"
        }
    
    def get_calendar_events(self, email: str, start_date: str, end_date: str) -> List[Dict]:
        """Mock function to get calendar events for an attendee"""
        # Mock events for demonstration
        mock_events = {
            "userone.amd@gmail.com": [
                {
                    "StartTime": "2025-07-24T09:00:00+05:30",
                    "EndTime": "2025-07-24T10:00:00+05:30",
                    "NumAttendees": 1,
                    "Attendees": ["userone.amd@gmail.com"],
                    "Summary": "Morning Standup"
                }
            ],
            "usertwo.amd@gmail.com": [
                {
                    "StartTime": "2025-07-24T10:00:00+05:30",
                    "EndTime": "2025-07-24T10:30:00+05:30",
                    "NumAttendees": 3,
                    "Attendees": ["userone.amd@gmail.com", "usertwo.amd@gmail.com", "userthree.amd@gmail.com"],
                    "Summary": "Team Meet"
                },
                {
                    "StartTime": "2025-07-24T14:00:00+05:30",
                    "EndTime": "2025-07-24T15:00:00+05:30",
                    "NumAttendees": 1,
                    "Attendees": ["usertwo.amd@gmail.com"],
                    "Summary": "Client Call"
                }
            ],
            "userthree.amd@gmail.com": [
                {
                    "StartTime": "2025-07-24T10:00:00+05:30",
                    "EndTime": "2025-07-24T10:30:00+05:30",
                    "NumAttendees": 3,
                    "Attendees": ["userone.amd@gmail.com", "usertwo.amd@gmail.com", "userthree.amd@gmail.com"],
                    "Summary": "Team Meet"
                },
                {
                    "StartTime": "2025-07-24T13:00:00+05:30",
                    "EndTime": "2025-07-24T14:00:00+05:30",
                    "NumAttendees": 1,
                    "Attendees": ["SELF"],
                    "Summary": "Lunch with Customers"
                }
            ]
        }
        
        return mock_events.get(email, [])
    
    def find_optimal_slot(self, attendees: List[str], duration_minutes: int, 
                         time_preference: str, start_date: str) -> Tuple[str, str]:
        """Find optimal meeting slot for all attendees"""
        # Parse start date
        try:
            base_date = date_parser.parse(start_date).date()
        except:
            base_date = datetime.now().date()
        
        # Define working hours (9 AM to 6 PM IST)
        working_start = 9
        working_end = 18
        
        # Get all attendees' calendars
        all_events = {}
        for attendee in attendees:
            events = self.get_calendar_events(
                attendee, 
                base_date.isoformat(), 
                (base_date + timedelta(days=7)).isoformat()
            )
            all_events[attendee] = events
        
        # Try to find slot for next 7 days
        for day_offset in range(7):
            search_date = base_date + timedelta(days=day_offset)
            
            # Skip weekends unless specified
            if search_date.weekday() >= 5 and time_preference == "flexible":
                continue
            
            # Check if this matches time preference
            if time_preference != "flexible":
                day_name = search_date.strftime('%A').lower()
                if time_preference.lower() not in day_name:
                    continue
            
            # Find free slots for this day
            optimal_slot = self._find_day_slot(
                all_events, search_date, duration_minutes, 
                working_start, working_end
            )
            
            if optimal_slot:
                return optimal_slot
        
        # Fallback: return a slot anyway
        fallback_date = base_date + timedelta(days=1)
        start_time = self.timezone.localize(
            datetime.combine(fallback_date, datetime.min.time().replace(hour=10, minute=30))
        )
        end_time = start_time + timedelta(minutes=duration_minutes)
        
        return start_time.isoformat(), end_time.isoformat()
    
    def _find_day_slot(self, all_events: Dict, search_date, duration_minutes: int,
                      working_start: int, working_end: int) -> Optional[Tuple[str, str]]:
        """Find available slot for a specific day"""
        
        # Create time slots (30-minute intervals)
        current_time = working_start
        
        while current_time < working_end:
            slot_start = self.timezone.localize(
                datetime.combine(search_date, datetime.min.time().replace(hour=int(current_time), minute=int((current_time % 1) * 60)))
            )
            slot_end = slot_start + timedelta(minutes=duration_minutes)
            
            # Check if slot fits in working hours
            if slot_end.hour < working_end:
                # Check if all attendees are free
                if self._is_slot_free(all_events, slot_start, slot_end):
                    return slot_start.isoformat(), slot_end.isoformat()
            
            current_time += 0.5  # 30-minute increments
        
        return None
    
    def _is_slot_free(self, all_events: Dict, slot_start: datetime, slot_end: datetime) -> bool:
        """Check if time slot is free for all attendees"""
        
        for attendee, events in all_events.items():
            for event in events:
                event_start = date_parser.parse(event['StartTime'])
                event_end = date_parser.parse(event['EndTime'])
                
                # Check for overlap
                if (slot_start < event_end and slot_end > event_start):
                    return False
        
        return True
    
    def process_meeting_request(self, input_data: Dict) -> Dict:
        """Main function to process a meeting request"""
        try:
            # Extract attendee emails
            attendee_emails = [input_data['From']]
            for attendee in input_data.get('Attendees', []):
                attendee_emails.append(attendee['email'])
            
            # Parse email content using LLM
            parsed_info = self.parse_email_content(
                input_data.get('EmailContent', ''),
                input_data.get('Subject', ''),
                attendee_emails
            )
            
            # Find optimal meeting slot
            start_time, end_time = self.find_optimal_slot(
                attendee_emails,
                parsed_info.get('meeting_duration_minutes', 60),
                parsed_info.get('time_preference', 'flexible'),
                input_data.get('Datetime', '2025-07-19T12:34:55')
            )
            
            # Generate attendee sections with their existing events
            attendee_sections = []
            
            for attendee_email in attendee_emails:
                # Get existing events for this attendee
                existing_events = self.get_calendar_events(
                    attendee_email,
                    date_parser.parse(start_time).date().isoformat(),
                    (date_parser.parse(start_time).date() + timedelta(days=1)).isoformat()
                )
                
                # Add the new meeting event
                new_event = {
                    "StartTime": start_time,
                    "EndTime": end_time,
                    "NumAttendees": len(attendee_emails),
                    "Attendees": attendee_emails,
                    "Summary": input_data.get('Subject', 'Meeting')
                }
                
                # Combine events
                all_events = existing_events + [new_event]
                
                attendee_sections.append({
                    "email": attendee_email,
                    "events": all_events
                })
            
            # Build final output
            output = {
                "Request_id": input_data.get('Request_id', ''),
                "Datetime": input_data.get('Datetime', ''),
                "Location": input_data.get('Location', ''),
                "From": input_data.get('From', ''),
                "Attendees": attendee_sections,
                "Subject": input_data.get('Subject', ''),
                "EmailContent": input_data.get('EmailContent', ''),
                "EventStart": start_time,
                "EventEnd": end_time,
                "Duration_mins": str(parsed_info.get('meeting_duration_minutes', 60)),
                "MetaData": {
                    "urgency": parsed_info.get('urgency', 'medium'),
                    "meeting_type": parsed_info.get('meeting_type', 'general'),
                    "time_preference": parsed_info.get('time_preference', 'flexible')
                }
            }
            
            return output
            
        except Exception as e:
            print(f"Error processing meeting request: {e}")
            # Return minimal valid response on error
            return {
                **input_data,
                "EventStart": "2025-07-24T10:30:00+05:30",
                "EventEnd": "2025-07-24T11:30:00+05:30",
                "Duration_mins": "60",
                "MetaData": {"error": str(e)}
            }

# Initialize the scheduler
scheduler = AgenticScheduler()

In [3]:
def your_meeting_assistant(data): 
    """
    Main function called by Flask server
    This implements the Agentic AI Scheduling Assistant
    """
    return scheduler.process_meeting_request(data)

In [4]:
@app.route('/receive', methods=['POST'])
def receive():
    data = request.get_json()
    print(f"\n Received: {json.dumps(data, indent=2)}")
    new_data = your_meeting_assistant(data)  # Your AI Meeting Assistant Function Call
    received_data.append(data)
    print(f"\n\n\n Sending:\n {json.dumps(new_data, indent=2)}")
    return jsonify(new_data)

def run_flask():
    app.run(host='0.0.0.0', port=5000)

In [7]:
# Start Flask in a background thread
Thread(target=run_flask, daemon=True).start()

 * Serving Flask app '__main__'
 * Debug mode: off
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


In [8]:
# Test the implementation before starting the server
print("🧪 Testing Agentic AI Scheduling Assistant...")

# Load sample input
with open('JSON_Samples/Input_Request.json', 'r') as f:
    test_input = json.load(f)

print("\n📧 Sample Input:")
print(json.dumps(test_input, indent=2))

# Test the assistant
print("\n🤖 Processing with AI Assistant...")
import time
start_time = time.time()
result = your_meeting_assistant(test_input)
end_time = time.time()

print(f"\n⏱️ Processing Time: {end_time - start_time:.3f} seconds")
print("\n📅 Generated Output:")
print(json.dumps(result, indent=2))

# Validate
required_fields = ['Request_id', 'EventStart', 'EventEnd', 'Duration_mins', 'Attendees']
missing = [f for f in required_fields if f not in result]

if missing:
    print(f"\n❌ Missing fields: {missing}")
else:
    print("\n✅ All required fields present!")
    print("🎯 Ready for submission!")

🧪 Testing Agentic AI Scheduling Assistant...

📧 Sample Input:
{
  "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
  "Datetime": "19-07-2025T12:34:55",
  "Location": "IISc Bangalore",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "usertwo.amd@gmail.com"
    },
    {
      "email": "userthree.amd@gmail.com"
    }
  ],
  "Subject": "Agentic AI Project Status Update",
  "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project."
}

🤖 Processing with AI Assistant...

⏱️ Processing Time: 0.253 seconds

📅 Generated Output:
{
  "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
  "Datetime": "19-07-2025T12:34:55",
  "Location": "IISc Bangalore",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "userone.amd@gmail.com",
      "events": [
        {
          "StartTime": "2025-07-24T09:00:00+05:30",
          "EndTime": "2025-07-24T10:00:00+05:30",
          "NumAttendees": 1,
          "